## Disney Dataset Data preprocessing and Analysis
sc

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

### loading data to df

In [0]:
pth="dbfs:/FileStore/shared_uploads/srkprasad1717@gmail.com/disney_plus_shows.csv"

In [0]:
df=spark.read.format("csv").options(header=True,inferSchema=True).load(pth)

In [0]:
df.show(2)

+---------+--------------------+--------------------+------+-----+-----+-----------+-----------------+-------+--------------------+----------+--------------------+--------------------+---------------+---------------+--------------------+---------+-----------+----------+
|  imdb_id|               title|                plot|  type|rated| year|released_at|         added_at|runtime|               genre|  director|              writer|              actors|       language|        country|              awards|metascore|imdb_rating|imdb_votes|
+---------+--------------------+--------------------+------+-----+-----+-----------+-----------------+-------+--------------------+----------+--------------------+--------------------+---------------+---------------+--------------------+---------+-----------+----------+
|tt0147800|10 Things I Hate ...|A pretty, popular...| movie|PG-13| 1999|31 Mar 1999|November 12, 2019| 97 min|Comedy, Drama, Ro...|Gil Junger|Karen McCullah, K...|Heath Ledger, Jul...|Eng

In [0]:
df1=df.select(col('year'),col('director'),col('imdb_votes'),col('title'),col('awards'),col('released_at'))
df1.show(50)


+---------+--------------------+----------+--------------------+--------------------+-----------+
|     year|            director|imdb_votes|               title|              awards|released_at|
+---------+--------------------+----------+--------------------+--------------------+-----------+
|     1999|          Gil Junger|   283,945|10 Things I Hate ...|2 wins & 13 nomin...|31 Mar 1999|
|    2018–|                 N/A|       124|101 Dalmatian Street|                 N/A|25 Mar 2019|
|     1996|       Stephen Herek|    97,785|      101 Dalmatians|Nominated for 1 G...|27 Nov 1996|
|     2002|Jim Kammerud, Bri...|     7,434|101 Dalmatians 2:...|5 wins & 10 nomin...|21 Jan 2003|
|     2000|          Kevin Lima|    33,444|      102 Dalmatians|Nominated for 1 O...|22 Nov 2000|
|     2011|        James Hayman|     4,933|12 Dates of Chris...|       1 nomination.|11 Dec 2011|
|     1954|   Richard Fleischer|    27,761|20,000 Leagues Un...|Won 2 Oscars. Ano...|20 Jul 1955|
|        G|Animation

### preprocessing year column

In [0]:
 df2 = df1.withColumn('newYr', when(length(df1.year) >= 4 , df1.year[0:4]).otherwise(0)).withColumn('yr',when(col('newYr').cast("int").isNotNull(), col('newYr')).otherwise(0))

In [0]:
df2.show(100)

+--------------------+--------------------+----------+--------------------+--------------------+-----------+-----+----+
|                year|            director|imdb_votes|               title|              awards|released_at|newYr|  yr|
+--------------------+--------------------+----------+--------------------+--------------------+-----------+-----+----+
|                1999|          Gil Junger|   283,945|10 Things I Hate ...|2 wins & 13 nomin...|31 Mar 1999| 1999|1999|
|               2018–|                 N/A|       124|101 Dalmatian Street|                 N/A|25 Mar 2019| 2018|2018|
|                1996|       Stephen Herek|    97,785|      101 Dalmatians|Nominated for 1 G...|27 Nov 1996| 1996|1996|
|                2002|Jim Kammerud, Bri...|     7,434|101 Dalmatians 2:...|5 wins & 10 nomin...|21 Jan 2003| 2002|2002|
|                2000|          Kevin Lima|    33,444|      102 Dalmatians|Nominated for 1 O...|22 Nov 2000| 2000|2000|
|                2011|        James Haym

In [0]:
df2=df2.drop('year','newYr')

In [0]:
df2.show(50)

+--------------------+----------+--------------------+--------------------+-----------+----+
|            director|imdb_votes|               title|              awards|released_at|  yr|
+--------------------+----------+--------------------+--------------------+-----------+----+
|          Gil Junger|   283,945|10 Things I Hate ...|2 wins & 13 nomin...|31 Mar 1999|1999|
|                 N/A|       124|101 Dalmatian Street|                 N/A|25 Mar 2019|2018|
|       Stephen Herek|    97,785|      101 Dalmatians|Nominated for 1 G...|27 Nov 1996|1996|
|Jim Kammerud, Bri...|     7,434|101 Dalmatians 2:...|5 wins & 10 nomin...|21 Jan 2003|2002|
|          Kevin Lima|    33,444|      102 Dalmatians|Nominated for 1 O...|22 Nov 2000|2000|
|        James Hayman|     4,933|12 Dates of Chris...|       1 nomination.|11 Dec 2011|2011|
|   Richard Fleischer|    27,761|20,000 Leagues Un...|Won 2 Oscars. Ano...|20 Jul 1955|1954|
|Animation, Advent...|       7.2|        A Bug's Life|                

### preprocessing votes column

In [0]:
df3 = df2.select(col('title'),col('yr').cast('int'),col('director'),col('awards'),col('released_at'), regexp_replace(col('imdb_votes'),',','').cast('int').alias('votes'))

### preprocessing release date column

In [0]:
df3=df3.withColumn('release_date',split(df1.released_at,' ')[2])
df3=df3.drop('released_at')

In [0]:
df3.show(150)

+--------------------+----+--------------------+--------------------+-------+------------+
|               title|  yr|            director|              awards|  votes|release_date|
+--------------------+----+--------------------+--------------------+-------+------------+
|10 Things I Hate ...|1999|          Gil Junger|2 wins & 13 nomin...| 283945|        1999|
|101 Dalmatian Street|2018|                 N/A|                 N/A|    124|        2019|
|      101 Dalmatians|1996|       Stephen Herek|Nominated for 1 G...|  97785|        1996|
|101 Dalmatians 2:...|2002|Jim Kammerud, Bri...|5 wins & 10 nomin...|   7434|        2003|
|      102 Dalmatians|2000|          Kevin Lima|Nominated for 1 O...|  33444|        2000|
|12 Dates of Chris...|2011|        James Hayman|       1 nomination.|   4933|        2011|
|20,000 Leagues Un...|1954|   Richard Fleischer|Won 2 Oscars. Ano...|  27761|        1955|
|        A Bug's Life|   0|Animation, Advent...|                 USA|      7|        null|

In [0]:
df3.printSchema()

root
 |-- title: string (nullable = true)
 |-- yr: integer (nullable = true)
 |-- director: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- votes: integer (nullable = true)



In [0]:
df4=df3.withColumn('releaseYr',when(col('release_date').cast("int").isNotNull(), col('release_date')).otherwise(0))
df4=df4.drop('release_date')
df4.printSchema()


root
 |-- title: string (nullable = true)
 |-- yr: integer (nullable = true)
 |-- director: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- votes: integer (nullable = true)
 |-- releaseYr: string (nullable = true)



In [0]:
df5=df4.select(col('title'),col('yr'),col('director'),col('awards'),col('votes'),col('releaseYr').cast('int').alias('release_year'))
df5.show(100)

+--------------------+----+--------------------+--------------------+-------+------------+
|               title|  yr|            director|              awards|  votes|release_year|
+--------------------+----+--------------------+--------------------+-------+------------+
|10 Things I Hate ...|1999|          Gil Junger|2 wins & 13 nomin...| 283945|        1999|
|101 Dalmatian Street|2018|                 N/A|                 N/A|    124|        2019|
|      101 Dalmatians|1996|       Stephen Herek|Nominated for 1 G...|  97785|        1996|
|101 Dalmatians 2:...|2002|Jim Kammerud, Bri...|5 wins & 10 nomin...|   7434|        2003|
|      102 Dalmatians|2000|          Kevin Lima|Nominated for 1 O...|  33444|        2000|
|12 Dates of Chris...|2011|        James Hayman|       1 nomination.|   4933|        2011|
|20,000 Leagues Un...|1954|   Richard Fleischer|Won 2 Oscars. Ano...|  27761|        1955|
|        A Bug's Life|   0|Animation, Advent...|                 USA|      7|           0|

In [0]:
df5.printSchema()

root
 |-- title: string (nullable = true)
 |-- yr: integer (nullable = true)
 |-- director: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- votes: integer (nullable = true)
 |-- release_year: integer (nullable = true)



In [0]:
display(df5)

title,yr,director,awards,votes,release_year
10 Things I Hate About You,1999,Gil Junger,2 wins & 13 nominations.,283945,1999
101 Dalmatian Street,2018,N/A,N/A,124,2019
101 Dalmatians,1996,Stephen Herek,Nominated for 1 Golden Globe. Another 3 wins & 9 nominations.,97785,1996
101 Dalmatians 2: Patch's London Adventure,2002,"Jim Kammerud, Brian Smith",5 wins & 10 nominations.,7434,2003
102 Dalmatians,2000,Kevin Lima,Nominated for 1 Oscar. Another 1 win & 7 nominations.,33444,2000
12 Dates of Christmas,2011,James Hayman,1 nomination.,4933,2011
"20,000 Leagues Under the Sea",1954,Richard Fleischer,Won 2 Oscars. Another 3 wins & 3 nominations.,27761,1955
A Bug's Life,0,"Animation, Adventure, Comedy, Family",USA,7,0
A Celebration of the Music from Coco,2020,Ron de Moraes,N/A,35,2020
A Goofy Movie,1995,Kevin Lima,5 nominations.,43655,1995


### 1. Which director's movies has the highest votes? and for that director, which year he has the highest votes?

In [0]:

df5.sort(col('votes').desc()).show(3)

+--------------------+----+--------------+--------------------+-------+------------+
|               title|  yr|      director|              awards|  votes|release_year|
+--------------------+----+--------------+--------------------+-------+------------+
|        The Avengers|2012|   Joss Whedon|Nominated for 1 O...|1225316|        2012|
|Star Wars: Episod...|1977|  George Lucas|Won 6 Oscars. Ano...|1188658|        1977|
|Star Wars: Episod...|1980|Irvin Kershner|Won 1 Oscar. Anot...|1109656|        1980|
+--------------------+----+--------------+--------------------+-------+------------+
only showing top 3 rows



In [0]:
df5.groupBy('director').agg(max('votes').alias('max_votes')).sort(col('max_votes').desc()).show()

+--------------------+---------+
|            director|max_votes|
+--------------------+---------+
|         Joss Whedon|  1225316|
|        George Lucas|  1188658|
|      Irvin Kershner|  1109656|
|       James Cameron|  1086714|
|          James Gunn|  1007917|
|      Andrew Stanton|   955757|
|    Richard Marquand|   912250|
|Andrew Stanton, L...|   911647|
|         Jon Favreau|   910300|
|Roger Allers, Rob...|   901362|
|Pete Docter, Bob ...|   895906|
|       John Lasseter|   846332|
|         J.J. Abrams|   833706|
|Pete Docter, Davi...|   779728|
|         Lee Unkrich|   729086|
|     Kenneth Branagh|   717145|
|        Joe Johnston|   711603|
|Anthony Russo, Jo...|   706555|
|      Gore Verbinski|   635041|
|           Brad Bird|   633057|
+--------------------+---------+
only showing top 20 rows



### 2. Total number of movies released in 1999, by director wise.

In [0]:

df5.filter((df5.release_year==1999) & (df5.director!="N/A")).groupBy('director').agg(count('title').alias('movies_count_in_1999')).sort(col('movies_count_in_1999').desc()).show(50)

+--------------------+--------------------+
|            director|movies_count_in_1999|
+--------------------+--------------------+
|     Kenneth Johnson|                   2|
|          Gil Junger|                   1|
|       Maurice Joyce|                   1|
|       David Kellogg|                   1|
|        LeVar Burton|                   1|
|Robert C. Ramirez...|                   1|
|          Rod Daniel|                   1|
|         David Lynch|                   1|
|         Steve Boyum|                   1|
|         Greg Beeman|                   1|
|      Duwayne Dunham|                   1|
|John Lasseter, As...|                   1|
|        Rob Marshall|                   1|
|      Paul Schneider|                   1|
|        George Lucas|                   1|
|Jun Falkenstein, ...|                   1|
|       Donald Petrie|                   1|
+--------------------+--------------------+



### 3. Which year movies got highest awards? And it belongs to which director?  ---pending

In [0]:
df3.select(col('awards')[instr(lower(col('awards')),'w')-2:instr(lower(col('awards')),'w')+2]).show(100)

In [0]:
df3.withColumn('b',instr(lower(col('awards')),'w')-2).withColumn('a', instr(lower(col('awards')),'w')+2).show()

### 4.Total number of movies nominated for awards between 1998-2018?

In [0]:
df5.filter((df5.yr>=1998) & (df5.yr<=2018) & (instr(lower(df5.awards), 'nominat' ) > 0)).show()

+--------------------+----+--------------------+--------------------+-------+------------+
|               title|  yr|            director|              awards|  votes|release_year|
+--------------------+----+--------------------+--------------------+-------+------------+
|10 Things I Hate ...|1999|          Gil Junger|2 wins & 13 nomin...| 283945|        1999|
|101 Dalmatians 2:...|2002|Jim Kammerud, Bri...|5 wins & 10 nomin...|   7434|        2003|
|      102 Dalmatians|2000|          Kevin Lima|Nominated for 1 O...|  33444|        2000|
|12 Dates of Chris...|2011|        James Hayman|       1 nomination.|   4933|        2011|
|A Ring of Endless...|2002|         Greg Beeman|2 wins & 1 nomina...|   1042|        2002|
|   A Wrinkle in Time|2018|        Ava DuVernay|5 wins & 15 nomin...|  38659|        2018|
| Alice in Wonderland|2010|          Tim Burton|Won 2 Oscars. Ano...| 371772|        2010|
|   Alley Cats Strike|2000|          Rod Daniel|       1 nomination.|   1840|        2000|

In [0]:
df5.filter((df5.yr>=1998) & (df5.yr<=2018) & (instr(lower(df5.awards), 'nominat' ) > 0)).count()

Out[43]: 348